In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [13]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [14]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [24]:
total = data_train['sales_value'].sum()
weighted_items = (data.groupby('item_id')['sales_value'].sum()/total).reset_index()

In [25]:
weighted_items = weighted_items.rename(columns={'sales_value': 'weight'})
weighted_items.head(2)

,item_id,weight
0,25671,2.969296e-06
1,26081,1.403822e-07


In [26]:
def weighted_random_recommendation(weighted_items, n=5):

    recs = weighted_items.sample(n, replace=False, weights='weight')
    
    return recs['item_id'].tolist()

In [27]:
%%time

weighted_random_recommendation(weighted_items, n=5)

Wall time: 20.1 ms


[995242, 1051516, 1053931, 1084437, 5591520]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [28]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[553111, 990130, 14025228, 8091299, 855288]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[8155825, 1841061, 946817, 1541203, 8203466]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [29]:
def p(v):
    return [int(id) for id in v[1:-1].split()]

result = pd.read_csv('predictions_basic.csv', converters={'actual': p, 'random_recommendation': eval, 'popular_recommendation': eval, 'itemitem': eval, 'cosine': eval, 'tfidf': eval, 'own_purchases': eval})
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[553111, 990130, 14025228, 8091299, 855288]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8155825, 1841061, 946817, 1541203, 8203466]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [30]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list[:k])
    
    precision = flags.sum() / k
    
    return precision

In [31]:
metrics = {}
cols = ['random_recommendation', 'popular_recommendation', 'itemitem', 'cosine', 'tfidf', 'own_purchases']

for i, r in result.iterrows():

    for c in cols:
        if not c in metrics:
            metrics[c] = []
        
        metrics[c].append(precision_at_k(r[c], r['actual']))

In [32]:
mdf = pd.DataFrame(metrics)
mdf['user_id'] = result['user_id']

In [33]:
mdf.head(5)

,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,user_id
0,0.0,0.2,0.4,0.2,0.2,0.4,1
1,0.0,0.0,0.0,0.0,0.0,0.0,3
2,0.0,0.0,0.2,0.0,0.0,0.0,6
3,0.0,0.2,0.2,0.4,0.4,0.2,7
4,0.0,0.4,0.2,0.2,0.2,0.4,8


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [9]:
# your_code